In [ ]:
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt
import vdmlab as vdm

In [ ]:
tuning = [np.array([0., 1., 0., 0., 0., 0., 0., 2., 0., 0., 0., 0.]), 
          np.array([0., 1., 2., 3., 3., 1., 0., 0., 0., 0., 0., 0.]), 
          np.array([0., 3., 3., 3., 0., 2., 0., 0., 3., 2., 0., 0.]),
          np.array([0., 1., 0., 0., 0., 0., 0., 4., 0., 0., 0., 0.])]

In [ ]:
for tc in tuning:
    plt.plot(tc)
    plt.axhline(2, color='k')
plt.show()

In [ ]:
all_fields = vdm.find_fields(tuning, hz_thresh=2, field_dist=1, max_mean_firing=10)

In [ ]:
all_fields

In [ ]:
hz_thresh=2

fields = []
for neuron_tc in tuning:
    neuron_field = np.zeros(neuron_tc.shape[0])
    for i, this_bin in enumerate(neuron_tc):
        if this_bin > hz_thresh:
            neuron_field[i] = 1
    fields.append(neuron_field)
fields

In [ ]:
tuning[0].shape

In [ ]:
def consecutive(array, stepsize=1):
    return np.split(array, np.where(np.diff(array) != stepsize)[0]+1)

fields_idx = dict()
for i, neuron_fields in enumerate(fields):
    field_idx = np.nonzero(neuron_fields)[0]
    fields_idx[i] = consecutive(field_idx)

In [ ]:
def consecutive(array):
    stepsize = 1
    return np.split(array, np.where(np.diff(array) != stepsize)[0]+1)

def find_fields(tuning, hz_thresh=5, min_length=1, max_length=20, max_mean_firing=10):

    fields = []
    for neuron_tc in tuning:
        if np.mean(neuron_tc < max_mean_firing):
            neuron_field = np.zeros(neuron_tc.shape[0])
            for i, this_bin in enumerate(neuron_tc):
                if this_bin > hz_thresh:
                    neuron_field[i] = 1
            fields.append(neuron_field)
        else:
            fields.append(np.array([]))
        
    fields_idx = dict()
    for i, neuron_fields in enumerate(fields):
        field_idx = np.nonzero(neuron_fields)[0]
        fields_idx[i] = consecutive(field_idx)
        print(type(fields_idx[i]))

    with_fields = dict()
    for key in fields_idx:
        for field in fields_idx[key]:
            if len(field) > max_length:
                continue
            elif min_length <= len(field):
                with_fields[key] = fields_idx[key]
                continue
    return with_fields

In [ ]:
with_fields = find_fields(tuning, hz_thresh=2, min_length=1, max_length=5, max_mean_firing=8)

In [ ]:
assert np.allclose(with_fields[1], [np.array([3, 4])])
assert np.allclose(with_fields[2][0], np.array([1, 2, 3]))
assert np.allclose(with_fields[2][1], np.array([8]))
assert np.allclose(with_fields[3], [np.array([7])])


In [ ]:
single_fields = vdm.get_single_field(with_fields)

In [ ]:
print(single_fields)

In [ ]:
np.mean(tuning[2])

In [ ]:
type(tuning[0])

In [ ]:
len(single_fields)

In [ ]:
linear = dict()
linear['position'] = np.concatenate((np.linspace(0, 10, 10), np.linspace(10, 0, 10), np.linspace(0, 10, 20)))
linear['time'] = np.linspace(0, 100, 40)

In [ ]:
plt.plot(linear['time'], linear['position'])
plt.show()

In [ ]:
import random
random.seed(0)
spikes = dict(time=[[], [], [], []])
for i in range(10):
    spikes['time'][0].append(random.uniform(0, 100))
for i in range(15):
    spikes['time'][1].append(random.uniform(20, 30))
for i in range(15):
    spikes['time'][2].append(random.uniform(60, 80))   
for i in range(15):
    spikes['time'][3].append(random.uniform(5, 10))
for i in range(5):
    for neuron_spikes in spikes['time']:
        neuron_spikes.append(random.uniform(0, 100))

In [ ]:
for idx, neuron_spikes in enumerate(spikes['time']):
    plt.plot(neuron_spikes, np.ones(len(neuron_spikes))+idx, '|', color='k', ms=10, mew=1)
plt.ylim(0.5, 4.5)
plt.show()

In [ ]:
tuning = vdm.tuning_curve(linear, spikes, sampling_rate=1, binsize=3, filter_type=None)

In [ ]:
colours = ['b', 'g', 'r', 'c']

x = list(range(0, np.shape(tuning)[1]))
for i, neuron_tc in enumerate(tuning):
    plt.plot(neuron_tc, color=colours[i])
#     plt.fill_between(x, 0, neuron_tc, facecolor=colours[i])
plt.show()

In [ ]:
tuning